In [ ]:
import requests
from fake_useragent import UserAgent
from bs4 import BeautifulSoup

In [ ]:
url = "https://www.bls.gov/news.release/cpi.htm"
ua = UserAgent()
header = {
    'User-Agent': ua.chrome
}

r = requests.get(url, headers=header)

In [ ]:
bs = BeautifulSoup(r.content, "html.parser")

In [ ]:
import datetime

update = bs.find(class_="update")
date = update.text.split(":")[1].strip()

datetime.datetime.strptime("November 1, 2021", "%B %d, %Y")

In [ ]:
table1 = bs.find(id="cpipress1")

In [ ]:
type(table1)

In [ ]:
table1.find(class_= "tableTitle").text

## tableのrow, colを計算、headerを作成

In [ ]:
thead = table1.find("thead")
max_row = 0
max_col = 0
header = [[]]
for row, tr in enumerate(thead.find_all("tr")):
    max_row = max(row + 1, max_row)
    offset = 0
    for col, th in enumerate(tr.find_all("th")):
        rspan = 1
        cspan = 1
        if "rowspan" in th.attrs:
            rspan = int(th["rowspan"])
        if "colspan" in th.attrs:
            cspan = int(th["colspan"])
        max_col = max(col + offset + cspan, max_col)
        max_row = max(row + rspan, max_row)
        offset += cspan - 1
        while not len(header) == max_row:
            header.append([])
        for r in range(row, row + rspan):
            for _ in range(col + offset, col + offset + cspan):
                header[r].append(th.text)

## tableのbodyを読み込み

In [ ]:
tbody = table1.find("tbody")
trows = tbody.find_all("tr")
data = []
for row, tr in enumerate(trows):
    data.append(["" for _ in range(max_col)])
    if "class" in tr.attrs and tr["class"][0] == "sep":
        continue
    th = tr.find("th")
    off = int(th.find("p")["class"][0].replace("sub", ""))
    data[row][0] = " " * off + th.text
    offset = 0
    for col, td in enumerate(tr.find_all("td")):
        colspan = 1
        if "colspan" in td.attrs:
            colspan = int(td["colspan"])
        for ci in range(col + offset, col + offset + colspan):
            data[row][ci + 1] = td.text
        offset += colspan - 1

In [ ]:
type(thead)

In [ ]:
data

In [ ]:
header + data